In [1]:
import jax
import jaxlib

print("jax version:", jax.__version__)
print("jaxlib version:", jaxlib.__version__)

import scvi
import h5py
import anndata as ad
import pandas as pd
import scipy.sparse as sp
import numpy as np

jax version: 0.4.34
jaxlib version: 0.4.34


/Users/smriti/opt/anaconda3/envs/scvi-env/lib/python3.12/site-packages/docrep/decorators.py:43: SyntaxWarning: 'param_atse_counts' is not a valid key!
  doc = func(self, args[0].__doc__, *args[1:], **kwargs)
/Users/smriti/opt/anaconda3/envs/scvi-env/lib/python3.12/site-packages/docrep/decorators.py:43: SyntaxWarning: 'param_junc_counts' is not a valid key!
  doc = func(self, args[0].__doc__, *args[1:], **kwargs)
/Users/smriti/opt/anaconda3/envs/scvi-env/lib/python3.12/site-packages/docrep/decorators.py:43: SyntaxWarning: 'param_categorical_covariate_keys' is not a valid key!
  doc = func(self, args[0].__doc__, *args[1:], **kwargs)


In [2]:
combined_adata = ad.read_h5ad('/Users/smriti/Documents/Research_Knowles_Lab/multivi_tools_splicing/ann_data/TMS_BRAINONLY_Combined_GE_ATSE.h5ad')

In [3]:
scvi.model.MULTIVISPLICE.setup_anndata(combined_adata, batch_key="modality", atse_counts='cell_by_cluster_matrix', junc_counts='cell_by_junction_matrix')

In [4]:

model = scvi.model.MULTIVISPLICE(
    combined_adata,
    n_genes=(combined_adata.var["modality"] == "Gene_Expression").sum(),
    n_junctions=(combined_adata.var["modality"] == "Splicing").sum(),
)

model.view_anndata_setup()

/Users/smriti/opt/anaconda3/envs/scvi-env/lib/python3.12/site-packages/torch/nn/init.py:452: UserWarning: Initializing zero-element tensors is a no-op
  warnings.warn("Initializing zero-element tensors is a no-op")


Anndata setup with scvi-tools version 1.2.0.

Setup via `MULTIVISPLICE.setup_anndata` with arguments:

{
│   'layer': None,
│   'atse_counts': 'cell_by_junction_matrix',
│   'junc_counts': 'cell_by_cluster_matrix',
│   'batch_key': 'modality',
│   'size_factor_key': None,
│   'categorical_covariate_keys': None,
│   'continuous_covariate_keys': None,
│   'protein_expression_obsm_key': None,
│   'protein_names_uns_key': None
}

         Summary Statistics         
┏━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━┓
┃     Summary Stat Key     ┃ Value ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━┩
│    n_atse_counts_key     │ 77042 │
│         n_batch          │   1   │
│         n_cells          │ 19942 │
│ n_extra_categorical_covs │   0   │
│ n_extra_continuous_covs  │   0   │
│    n_junc_counts_key     │ 77042 │
│         n_labels         │   1   │
│          n_vars          │ 77042 │
└──────────────────────────┴───────┘

                        Data Registry                        
┏━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃  Registry Key   ┃           scvi-tools Location           ┃
┡━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│        X        │                 adata.X                 │
│ atse_counts_key │ adata.layers['cell_by_junction_matrix'] │
│      batch      │        adata.obs['_scvi_batch']         │
│      ind_x      │          adata.obs['_indices']          │
│ junc_counts_key │ adata.layers['cell_by_cluster_matrix']  │
│     labels      │        adata.obs['_scvi_labels']        │
└─────────────────┴─────────────────────────────────────────┘

                    batch State Registry                    
┏━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┓
┃    Source Location    ┃ Categories ┃ scvi-tools Encoding ┃
┡━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━┩
│ adata.obs['modality'] │   paired   │          0          │
└───────────────────────┴────────────┴─────────────────────┘

                     labels State Registry                      
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┓
┃      Source Location      ┃ Categories ┃ scvi-tools Encoding ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━┩
│ adata.obs['_scvi_labels'] │     0      │          0          │
└───────────────────────────┴────────────┴─────────────────────┘

                    batch State Registry                    
┏━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┓
┃    Source Location    ┃ Categories ┃ scvi-tools Encoding ┃
┡━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━┩
│ adata.obs['modality'] │   paired   │          0          │
└───────────────────────┴────────────┴─────────────────────┘